In [1]:
import os
%pwd

'/media/sasan/Common/ML/EtoE/research'

In [2]:
os.chdir("../")
%pwd

'/media/sasan/Common/ML/EtoE'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_dir: Path
    all_schema: dict

In [10]:
from RedWineQualityPrediction.constanst import *
from RedWineQualityPrediction.utils.common import read_yaml, create_derectories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_derectories([self.config.artifacts_root])
        
    def data_validation_config(self) -> DataValidationConfig:
        confing = self.config.data_validation
        schema = self.schema.COLUMNS
        
        create_derectories([confing.root_dir])
        
        return DataValidationConfig(
            root_dir=confing.root_dir,
            STATUS_FILE=confing.STATUS_FILE,
            unzip_dir=confing.unzip_data_dir,
            all_schema=schema
        )

In [7]:
import os
from RedWineQualityPrediction import logger
import pandas as pd

In [24]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        
    def validate_all_columns(self)-> bool:
        try:
            validation_status = None
            
            dataset = pd.read_csv(self.config.unzip_dir)
            all_cols = list(dataset.columns)
            
            all_schemas = self.config.all_schema.keys()
            
            for col in all_cols:
                if col not in all_schemas:
                    validation_status=False
                    logger.info(f"ERRO: {col} is not in schema")
                else:
                    validation_status=True
            if validation_status:
                # Validate the columns datatypes
                    expected_schema = self.config.all_schema # get key and value both pares
                    
                    for col, expec_dtype in expected_schema.items():
                        dataset_col_dtype = dataset[col].dtype
                        
                        # compare the csv file data type and schema data type
                        if dataset_col_dtype != expec_dtype :
                            validation_status=False
                            logger.info(f"ERRO: {col} data type is {dataset_col_dtype} and schema data type is {expec_dtype}")
                        else:
                            validation_status=True
                            logger.info(f"Data validation success in :: {col}")
                    
            with open(self.config.STATUS_FILE, "w") as f:
                        f.write(f"Validation Status: {validation_status}")
                        
            return validation_status
            
        except Exception as error:
            raise error

In [23]:
try:
    config = ConfigurationManager()
    data_val_config = config.data_validation_config()
    data_val = DataValidation(config=data_val_config)
    data_val.validate_all_columns()
except Exception as error:
    raise error

[2025-02-06 13:59:53,162: INFO: common: Reading the yaml file from ::     config/config.yml]
[2025-02-06 13:59:53,167: INFO: common: Reading the yaml file from ::     params.yml]
[2025-02-06 13:59:53,174: INFO: common: Reading the yaml file from ::     schema.yml]
[2025-02-06 13:59:53,176: INFO: common: Directory created at :: artifacts]
[2025-02-06 13:59:53,178: INFO: common: Directory created at :: artifacts/data_validation]
[2025-02-06 13:59:53,189: INFO: 3451669464: Data validation success in :: fixed acidity]
[2025-02-06 13:59:53,191: INFO: 3451669464: Data validation success in :: volatile acidity]
[2025-02-06 13:59:53,193: INFO: 3451669464: Data validation success in :: citric acid]
[2025-02-06 13:59:53,195: INFO: 3451669464: Data validation success in :: residual sugar]
[2025-02-06 13:59:53,197: INFO: 3451669464: Data validation success in :: chlorides]
[2025-02-06 13:59:53,198: INFO: 3451669464: Data validation success in :: free sulfur dioxide]
[2025-02-06 13:59:53,200: INFO: